In [3]:
import pandas as pd
import requests
import json
import numpy as np
import geopandas as gpd

## Funkcje

In [4]:
def get_df (page_url):
    response_API = requests.get(page_url)
    population_summary = json.loads(response_API.text)
    rows = []

    for data in population_summary['results']:
        for value in data['values']:
            row = {'id': data['id'],
                'name': data['name'],
                'year': value['year'],
                'value': value['val'],
                'powiat_numer': data['id'][2:4] + data['id'][7:9]}
            rows.append(row)
    df = pd.DataFrame(rows)
    return df

In [5]:
def transform_population_df(population_df):
    population_df_copy = population_df.copy()
    population_df_copy['name_normalized'] = population_df['name'].str.replace('Powiat', '')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace(' m.', '')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.lstrip()
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace('Wałbrzych od 2013', 'Wałbrzych')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace('Wałbrzych do 2002', 'Wałbrzych')
    population_df_copy['name_normalized'] = population_df_copy['name_normalized'].str.replace('st. Warszawa', 'Warszawa')
    population_df_copy['powiat_numer'] = population_df_copy['powiat_numer'].astype('int').astype('str')
    population_df_copy = population_df_copy.rename(columns={'value':'population'})
    return population_df_copy

In [6]:
def get_total_df(deaths_path):
    ALL_DATA = pd.read_csv(deaths_path)
    poviats_data_Y = ALL_DATA.query('powiat_numer > 32')
    total_df = poviats_data_Y.query(f'wiek_eurostat != "TOTAL"')
    total_df_copy = total_df.copy()
    total_df_copy['name_normalized'] = total_df_copy['powiat_nazwa']
    total_df_copy = total_df_copy.rename(columns={'YEAR':'year', 'sum':'deaths'})
    total_df_copy['year'] = total_df_copy['year'].astype('str')
    total_df_copy['powiat_numer'] = total_df_copy['powiat_numer'].astype('str') 

    return total_df_copy

In [7]:
def get_CMR_table(population_df, total_df):
    merged_df = pd.merge(population_df, total_df, on=['year', 'powiat_numer', 'age_group'], how='inner')
    merged_df = merged_df.rename(columns={'name_normalized_x':'name_normalized_pop', 'name_normalized_y':'name_normalized_tot'})
    # merged_df['CMR'] = merged_df['deaths']/merged_df['population']*100000

    return merged_df

## Dla grupy ogółem

In [1]:
age_groups = """Y_LT5    72306
Y5-9    72307
Y10-14    72308
Y15-19    72309
Y20-24    47734
Y25-29    47694
Y30-34    47722
Y35-39    47701
Y40-44    47707
Y45-49    47726
Y50-54    47717
Y55-59    47732
Y60-64    47739
Y65-69    72239
Y70-74    76022
Y75-79    76023
Y80-84    76024
Y_GE85    76025"""

# Split the data into lines and then split each line into key and value
lines = age_groups.split("\n")
keys = []
values = []

for line in lines:
    key, value = line.split()
    keys.append(key)
    values.append(int(value))

# Create a dictionary from the keys and values lists
age_dict = dict(zip(keys, values))
print(age_dict)

{'Y_LT5': 72306, 'Y5-9': 72307, 'Y10-14': 72308, 'Y15-19': 72309, 'Y20-24': 47734, 'Y25-29': 47694, 'Y30-34': 47722, 'Y35-39': 47701, 'Y40-44': 47707, 'Y45-49': 47726, 'Y50-54': 47717, 'Y55-59': 47732, 'Y60-64': 47739, 'Y65-69': 72239, 'Y70-74': 76022, 'Y75-79': 76023, 'Y80-84': 76024, 'Y_GE85': 76025}


In [15]:
population_df = pd.DataFrame()

for k, v in age_dict.items():
    current_df = pd.DataFrame()
    page_0 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=0'
    page_1 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=1'
    page_2 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=2'
    page_3 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=3'
    df_0 = get_df(page_0)
    df_1 = get_df(page_1)
    df_2 = get_df(page_2)
    df_3 = get_df(page_3)
    dfs = [df_0, df_1, df_2, df_3]
    current_df = pd.concat(dfs, axis=0)
    current_df['age_group'] = str(k)
    population_df = pd.concat([population_df, current_df], axis=0)


In [16]:
population_df

,id,name,year,value,powiat_numer,age_group
0,011212001000,Powiat bocheński,2000,6232,1201,Y_LT5
1,011212001000,Powiat bocheński,2001,6020,1201,Y_LT5
2,011212001000,Powiat bocheński,2002,5842,1201,Y_LT5
3,011212001000,Powiat bocheński,2003,5704,1201,Y_LT5
4,011212001000,Powiat bocheński,2004,5588,1201,Y_LT5
...,...,...,...,...,...,...
1615,071427338000,Powiat żyrardowski,2017,1482,1438,Y_GE85
1616,071427338000,Powiat żyrardowski,2018,1467,1438,Y_GE85
1617,071427338000,Powiat żyrardowski,2019,1436,1438,Y_GE85
1618,071427338000,Powiat żyrardowski,2020,1392,1438,Y_GE85


Komentarz: dane dla osob 70+ są dostępne tylko od roku 2002

In [20]:
population_df.groupby('age_group').count()

,id,name,year,value,powiat_numer
age_group,,,,,
Y10-14,8336,8336,8336,8336,8336
Y15-19,8336,8336,8336,8336,8336
Y20-24,8336,8336,8336,8336,8336
Y25-29,8336,8336,8336,8336,8336
Y30-34,8336,8336,8336,8336,8336
Y35-39,8336,8336,8336,8336,8336
Y40-44,8336,8336,8336,8336,8336
Y45-49,8336,8336,8336,8336,8336
Y5-9,8336,8336,8336,8336,8336


In [22]:
population_df = transform_population_df(population_df=population_df)

In [23]:
population_df

,id,name,year,population,powiat_numer,age_group,name_normalized
0,011212001000,Powiat bocheński,2000,6232,1201,Y_LT5,bocheński
1,011212001000,Powiat bocheński,2001,6020,1201,Y_LT5,bocheński
2,011212001000,Powiat bocheński,2002,5842,1201,Y_LT5,bocheński
3,011212001000,Powiat bocheński,2003,5704,1201,Y_LT5,bocheński
4,011212001000,Powiat bocheński,2004,5588,1201,Y_LT5,bocheński
...,...,...,...,...,...,...,...
1615,071427338000,Powiat żyrardowski,2017,1482,1438,Y_GE85,żyrardowski
1616,071427338000,Powiat żyrardowski,2018,1467,1438,Y_GE85,żyrardowski
1617,071427338000,Powiat żyrardowski,2019,1436,1438,Y_GE85,żyrardowski
1618,071427338000,Powiat żyrardowski,2020,1392,1438,Y_GE85,żyrardowski


In [24]:
deaths_path = "/Users/nelatoma/Documents/icm/magisterka/zgony/MAGISTERKA_OFFICIAL/results/all_poviats_stats.csv"

In [27]:
total_df = get_total_df(deaths_path=deaths_path)

In [28]:
total_df

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
392,0 - 4,201,bolesławiecki,2000,Y_LT5,6,bolesławiecki
393,0 - 4,202,dzierżoniowski,2000,Y_LT5,10,dzierżoniowski
394,0 - 4,203,głogowski,2000,Y_LT5,7,głogowski
395,0 - 4,204,górowski,2000,Y_LT5,4,górowski
396,0 - 4,205,jaworski,2000,Y_LT5,4,jaworski
...,...,...,...,...,...,...,...
174215,90 i więcej,3217,wałecki,2021,Y_GE85,81,wałecki
174216,90 i więcej,3218,łobeski,2021,Y_GE85,68,łobeski
174217,90 i więcej,3261,Koszalin,2021,Y_GE85,219,Koszalin
174218,90 i więcej,3262,Szczecin,2021,Y_GE85,956,Szczecin


Warszawski i Warszawa w roku 2002, zlozone do kupy do Warszawy

In [62]:
total_df.loc[((total_df['name_normalized'] == 'Warszawa' ) & (total_df['year'] == '2002')), 'deaths'] = total_df.loc[((total_df['name_normalized'] == 'Warszawa' ) & (total_df['year'] == '2002')), 'deaths'].add(total_df.loc[((total_df['name_normalized'] == 'warszawski' ) & (total_df['year'] == '2002')), 'deaths'].values)

In [66]:
rows_to_drop = total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

In [67]:
total_df.drop(rows_to_drop.index, inplace=True)

In [70]:
total_df.query("name_normalized == 'warszawski' and year == '2002'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized


In [71]:
population_df.query("name_normalized == 'warszawski' and year == '2002'")

,id,name,year,population,powiat_numer,age_group,name_normalized


Pare renamow

In [77]:
total_df = total_df.rename(columns={'wiek_eurostat':'age_group'})

Grupy wiekowe 84-89 i 90 i wiecej zlozone razem

In [76]:
total_df.query("wiek_eurostat == 'Y_GE85' and name_normalized == 'bolesławiecki'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
7022,85 - 89,201,bolesławiecki,2000,Y_GE85,63,bolesławiecki
7412,90 i więcej,201,bolesławiecki,2000,Y_GE85,33,bolesławiecki
14822,85 - 89,201,bolesławiecki,2001,Y_GE85,52,bolesławiecki
15212,90 i więcej,201,bolesławiecki,2001,Y_GE85,38,bolesławiecki
22766,85 - 89,201,bolesławiecki,2002,Y_GE85,52,bolesławiecki
23164,90 i więcej,201,bolesławiecki,2002,Y_GE85,31,bolesławiecki
30690,85 - 89,201,bolesławiecki,2003,Y_GE85,42,bolesławiecki
31086,90 i więcej,201,bolesławiecki,2003,Y_GE85,41,bolesławiecki
38610,85 - 89,201,bolesławiecki,2004,Y_GE85,46,bolesławiecki
39006,90 i więcej,201,bolesławiecki,2004,Y_GE85,46,bolesławiecki


Join

In [95]:
merged_df = get_CMR_table(population_df, total_df)

In [96]:
merged_df.query("age_group == 'Y_GE85' and name_normalized_pop == 'bolesławiecki'")

,id,name,year,population,powiat_numer,age_group,name_normalized_pop,wiek,powiat_nazwa,deaths,name_normalized_tot
144594,030210101000,Powiat bolesławiecki,2002,467,201,Y_GE85,bolesławiecki,85 - 89,bolesławiecki,52,bolesławiecki
144595,030210101000,Powiat bolesławiecki,2002,467,201,Y_GE85,bolesławiecki,90 i więcej,bolesławiecki,31,bolesławiecki
144596,030210101000,Powiat bolesławiecki,2003,451,201,Y_GE85,bolesławiecki,85 - 89,bolesławiecki,42,bolesławiecki
144597,030210101000,Powiat bolesławiecki,2003,451,201,Y_GE85,bolesławiecki,90 i więcej,bolesławiecki,41,bolesławiecki
144598,030210101000,Powiat bolesławiecki,2004,499,201,Y_GE85,bolesławiecki,85 - 89,bolesławiecki,46,bolesławiecki
144599,030210101000,Powiat bolesławiecki,2004,499,201,Y_GE85,bolesławiecki,90 i więcej,bolesławiecki,46,bolesławiecki
144600,030210101000,Powiat bolesławiecki,2005,552,201,Y_GE85,bolesławiecki,85 - 89,bolesławiecki,61,bolesławiecki
144601,030210101000,Powiat bolesławiecki,2005,552,201,Y_GE85,bolesławiecki,90 i więcej,bolesławiecki,41,bolesławiecki
144602,030210101000,Powiat bolesławiecki,2006,579,201,Y_GE85,bolesławiecki,85 - 89,bolesławiecki,47,bolesławiecki
144603,030210101000,Powiat bolesławiecki,2006,579,201,Y_GE85,bolesławiecki,90 i więcej,bolesławiecki,41,bolesławiecki


In [85]:
len(total_df)

158384

In [86]:
len(population_df)

147064

In [97]:
len(merged_df)

154654

In [88]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154654 entries, 0 to 154653
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   154654 non-null  object 
 1   name                 154654 non-null  object 
 2   year                 154654 non-null  object 
 3   population           154654 non-null  int64  
 4   powiat_numer         154654 non-null  object 
 5   age_group            154654 non-null  object 
 6   name_normalized_pop  154654 non-null  object 
 7   wiek                 154654 non-null  object 
 8   powiat_nazwa         154654 non-null  object 
 9   deaths               154654 non-null  int64  
 10  name_normalized_tot  154654 non-null  object 
 11  CMR                  154654 non-null  float64
dtypes: float64(1), int64(2), object(9)
memory usage: 14.2+ MB


In [117]:
grouped_df = pd.DataFrame(merged_df.groupby(['name_normalized_tot', 'powiat_numer', 'year', 'age_group'], as_index=False).sum(['population', 'deaths']))

In [118]:
grouped_df

,name_normalized_tot,powiat_numer,year,age_group,population,deaths
0,Biała Podlaska,661,2000,Y10-14,5248,0
1,Biała Podlaska,661,2000,Y15-19,5933,4
2,Biała Podlaska,661,2000,Y20-24,4701,5
3,Biała Podlaska,661,2000,Y25-29,4198,8
4,Biała Podlaska,661,2000,Y30-34,3680,4
...,...,...,...,...,...,...
147059,żywiecki,2417,2021,Y70-74,7149,297
147060,żywiecki,2417,2021,Y75-79,4059,252
147061,żywiecki,2417,2021,Y80-84,3242,314
147062,żywiecki,2417,2021,Y_GE85,5836,596


In [116]:
grouped_df.query('age_group=="Y_GE85"')

,name_normalized_tot,powiat_numer,year,age_group,id,name,population,name_normalized_pop,wiek,powiat_nazwa,deaths
44,Biała Podlaska,661,2002,Y_GE85,060610961000060610961000,Powiat m. Biała PodlaskaPowiat m. Biała Podlaska,696,Biała PodlaskaBiała Podlaska,85 - 8990 i więcej,Biała PodlaskaBiała Podlaska,58
62,Biała Podlaska,661,2003,Y_GE85,060610961000060610961000,Powiat m. Biała PodlaskaPowiat m. Biała Podlaska,636,Biała PodlaskaBiała Podlaska,85 - 8990 i więcej,Biała PodlaskaBiała Podlaska,83
80,Biała Podlaska,661,2004,Y_GE85,060610961000060610961000,Powiat m. Biała PodlaskaPowiat m. Biała Podlaska,660,Biała PodlaskaBiała Podlaska,85 - 8990 i więcej,Biała PodlaskaBiała Podlaska,55
98,Biała Podlaska,661,2005,Y_GE85,060610961000060610961000,Powiat m. Biała PodlaskaPowiat m. Biała Podlaska,716,Biała PodlaskaBiała Podlaska,85 - 8990 i więcej,Biała PodlaskaBiała Podlaska,54
116,Biała Podlaska,661,2006,Y_GE85,060610961000060610961000,Powiat m. Biała PodlaskaPowiat m. Biała Podlaska,738,Biała PodlaskaBiała Podlaska,85 - 8990 i więcej,Biała PodlaskaBiała Podlaska,63
...,...,...,...,...,...,...,...,...,...,...,...
146990,żywiecki,2417,2017,Y_GE85,012414417000012414417000,Powiat żywieckiPowiat żywiecki,5848,żywieckiżywiecki,85 - 8990 i więcej,żywieckiżywiecki,470
147008,żywiecki,2417,2018,Y_GE85,012414417000012414417000,Powiat żywieckiPowiat żywiecki,6074,żywieckiżywiecki,85 - 8990 i więcej,żywieckiżywiecki,465
147026,żywiecki,2417,2019,Y_GE85,012414417000012414417000,Powiat żywieckiPowiat żywiecki,6170,żywieckiżywiecki,85 - 8990 i więcej,żywieckiżywiecki,472
147044,żywiecki,2417,2020,Y_GE85,012414417000012414417000,Powiat żywieckiPowiat żywiecki,5956,żywieckiżywiecki,85 - 8990 i więcej,żywieckiżywiecki,653


In [123]:
grouped_df['CMR'] = grouped_df['deaths']/grouped_df['population'] * 100000

In [126]:
grouped_df.query("age_group == 'Y_GE85' and name_normalized_tot == 'Biała Podlaska'")

,name_normalized_tot,powiat_numer,year,age_group,population,deaths,CMR
44,Biała Podlaska,661,2002,Y_GE85,696,58,8333.333333
62,Biała Podlaska,661,2003,Y_GE85,636,83,13050.314465
80,Biała Podlaska,661,2004,Y_GE85,660,55,8333.333333
98,Biała Podlaska,661,2005,Y_GE85,716,54,7541.899441
116,Biała Podlaska,661,2006,Y_GE85,738,63,8536.585366
134,Biała Podlaska,661,2007,Y_GE85,822,49,5961.070560
152,Biała Podlaska,661,2008,Y_GE85,918,56,6100.217865
170,Biała Podlaska,661,2009,Y_GE85,1008,74,7341.269841
188,Biała Podlaska,661,2010,Y_GE85,1076,85,7899.628253
206,Biała Podlaska,661,2011,Y_GE85,1184,80,6756.756757


In [127]:
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147064 entries, 0 to 147063
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name_normalized_tot  147064 non-null  object 
 1   powiat_numer         147064 non-null  object 
 2   year                 147064 non-null  object 
 3   age_group            147064 non-null  object 
 4   population           147064 non-null  int64  
 5   deaths               147064 non-null  int64  
 6   CMR                  147064 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 7.9+ MB


In [128]:
grouped_df.to_csv('../results/CMR_age_groups_Y.csv', index=False)

### Połączenie z dotychczasowymi danymi TOTAL dla latwiejszej wizualizacji

In [129]:
cmr_total_df = pd.read_csv('../results/CMR_total_Y.csv')

In [131]:
cmr_total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8336 entries, 0 to 8335
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   8336 non-null   int64  
 1   name                 8336 non-null   object 
 2   year                 8336 non-null   int64  
 3   population           8336 non-null   int64  
 4   name_normalized_pop  8336 non-null   object 
 5   powiat_numer         8336 non-null   int64  
 6   wiek                 8336 non-null   object 
 7   powiat_nazwa         8336 non-null   object 
 8   wiek_eurostat        8336 non-null   object 
 9   deaths               8336 non-null   int64  
 10  name_normalized_tot  8336 non-null   object 
 11  CMR                  8336 non-null   float64
dtypes: float64(1), int64(5), object(6)
memory usage: 781.6+ KB


In [132]:
cmr_total_copy_df = cmr_total_df.copy()

In [139]:
cmr_total_copy_df['year'] = cmr_total_copy_df['year'].astype('str')
cmr_total_copy_df['powiat_numer'] = cmr_total_copy_df['powiat_numer'].astype('str')
cmr_total_copy_df = cmr_total_copy_df.rename(columns={'wiek_eurostat':'age_group'})

In [140]:
cmr_total_copy_df.columns

Index(['id', 'name', 'year', 'population', 'name_normalized_pop',
       'powiat_numer', 'wiek', 'powiat_nazwa', 'age_group', 'deaths',
       'name_normalized_tot', 'CMR'],
      dtype='object')

In [137]:
grouped_df.columns

Index(['name_normalized_tot', 'powiat_numer', 'year', 'age_group',
       'population', 'deaths', 'CMR'],
      dtype='object')

In [142]:
FINAL_DF = pd.concat([cmr_total_copy_df[grouped_df.columns], grouped_df])

In [143]:
FINAL_DF

,name_normalized_tot,powiat_numer,year,age_group,population,deaths,CMR
0,bocheński,1201,2000,TOTAL,97328,837,859.978629
1,bocheński,1201,2001,TOTAL,97602,863,884.203193
2,bocheński,1201,2002,TOTAL,97962,896,914.640371
3,bocheński,1201,2003,TOTAL,98394,840,853.710592
4,bocheński,1201,2004,TOTAL,100063,866,865.454763
...,...,...,...,...,...,...,...
147059,żywiecki,2417,2021,Y70-74,7149,297,4154.427193
147060,żywiecki,2417,2021,Y75-79,4059,252,6208.425721
147061,żywiecki,2417,2021,Y80-84,3242,314,9685.379395
147062,żywiecki,2417,2021,Y_GE85,5836,596,10212.474297


In [144]:
len(cmr_total_copy_df) + len(grouped_df)

155400

In [147]:
FINAL_DF.query('name_normalized_tot == "Warszawa" and age_group == "Y_GE85"')

,name_normalized_tot,powiat_numer,year,age_group,population,deaths,CMR
20968,Warszawa,1465,2002,Y_GE85,42404,3580,8442.599755
20986,Warszawa,1465,2003,Y_GE85,40808,3667,8985.983141
21004,Warszawa,1465,2004,Y_GE85,42146,3460,8209.557253
21022,Warszawa,1465,2005,Y_GE85,44692,3568,7983.531728
21040,Warszawa,1465,2006,Y_GE85,47662,3459,7257.353867
21058,Warszawa,1465,2007,Y_GE85,52092,3640,6987.637257
21076,Warszawa,1465,2008,Y_GE85,57746,3893,6741.592491
21094,Warszawa,1465,2009,Y_GE85,62520,4291,6863.403711
21112,Warszawa,1465,2010,Y_GE85,65456,4436,6777.071621
21130,Warszawa,1465,2011,Y_GE85,70842,4680,6606.250529


In [148]:
FINAL_DF.to_csv('../results/ALL_CMR_Y.csv', index=False)

# Dla kobiet

In [149]:
age_groups = """Y_LT5	72296
Y5-9	72297
Y10-14	72298
Y15-19	72299
Y20-24	47738
Y25-29	47696
Y30-34	47695
Y35-39	47716
Y40-44	47698
Y45-49	47727
Y50-54	47723
Y55-59	47702
Y60-64	47693
Y65-69	72241
Y70-74	76014
Y75-79	76015
Y80-84	76016
Y_GE85	76017"""

# Split the data into lines and then split each line into key and value
lines = age_groups.split("\n")
keys = []
values = []

for line in lines:
    key, value = line.split()
    keys.append(key)
    values.append(int(value))

# Create a dictionary from the keys and values lists
age_dict = dict(zip(keys, values))
print(age_dict)

{'Y_LT5': 72296, 'Y5-9': 72297, 'Y10-14': 72298, 'Y15-19': 72299, 'Y20-24': 47738, 'Y25-29': 47696, 'Y30-34': 47695, 'Y35-39': 47716, 'Y40-44': 47698, 'Y45-49': 47727, 'Y50-54': 47723, 'Y55-59': 47702, 'Y60-64': 47693, 'Y65-69': 72241, 'Y70-74': 76014, 'Y75-79': 76015, 'Y80-84': 76016, 'Y_GE85': 76017}


In [150]:
population_df = pd.DataFrame()

for k, v in age_dict.items():
    current_df = pd.DataFrame()
    page_0 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=0'
    page_1 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=1'
    page_2 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=2'
    page_3 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=3'
    df_0 = get_df(page_0)
    df_1 = get_df(page_1)
    df_2 = get_df(page_2)
    df_3 = get_df(page_3)
    dfs = [df_0, df_1, df_2, df_3]
    current_df = pd.concat(dfs, axis=0)
    current_df['age_group'] = str(k)
    population_df = pd.concat([population_df, current_df], axis=0)

In [151]:
population_df

,id,name,year,value,powiat_numer,age_group
0,011212001000,Powiat bocheński,2000,3021,1201,Y_LT5
1,011212001000,Powiat bocheński,2001,2915,1201,Y_LT5
2,011212001000,Powiat bocheński,2002,2820,1201,Y_LT5
3,011212001000,Powiat bocheński,2003,2797,1201,Y_LT5
4,011212001000,Powiat bocheński,2004,2749,1201,Y_LT5
...,...,...,...,...,...,...
1615,071427338000,Powiat żyrardowski,2017,1091,1438,Y_GE85
1616,071427338000,Powiat żyrardowski,2018,1074,1438,Y_GE85
1617,071427338000,Powiat żyrardowski,2019,1067,1438,Y_GE85
1618,071427338000,Powiat żyrardowski,2020,1040,1438,Y_GE85


In [152]:
population_df = transform_population_df(population_df=population_df)


In [153]:
population_df

,id,name,year,population,powiat_numer,age_group,name_normalized
0,011212001000,Powiat bocheński,2000,3021,1201,Y_LT5,bocheński
1,011212001000,Powiat bocheński,2001,2915,1201,Y_LT5,bocheński
2,011212001000,Powiat bocheński,2002,2820,1201,Y_LT5,bocheński
3,011212001000,Powiat bocheński,2003,2797,1201,Y_LT5,bocheński
4,011212001000,Powiat bocheński,2004,2749,1201,Y_LT5,bocheński
...,...,...,...,...,...,...,...
1615,071427338000,Powiat żyrardowski,2017,1091,1438,Y_GE85,żyrardowski
1616,071427338000,Powiat żyrardowski,2018,1074,1438,Y_GE85,żyrardowski
1617,071427338000,Powiat żyrardowski,2019,1067,1438,Y_GE85,żyrardowski
1618,071427338000,Powiat żyrardowski,2020,1040,1438,Y_GE85,żyrardowski


In [154]:
deaths_path = "/Users/nelatoma/Documents/icm/magisterka/zgony/MAGISTERKA_OFFICIAL/results/all_poviats_stats_females.csv"

In [155]:
total_df = get_total_df(deaths_path=deaths_path)

In [156]:
total_df

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
392,0 - 4,201,bolesławiecki,2000,Y_LT5,3,bolesławiecki
393,0 - 4,202,dzierżoniowski,2000,Y_LT5,4,dzierżoniowski
394,0 - 4,203,głogowski,2000,Y_LT5,3,głogowski
395,0 - 4,204,górowski,2000,Y_LT5,2,górowski
396,0 - 4,205,jaworski,2000,Y_LT5,0,jaworski
...,...,...,...,...,...,...,...
174215,90 i więcej,3217,wałecki,2021,Y_GE85,62,wałecki
174216,90 i więcej,3218,łobeski,2021,Y_GE85,48,łobeski
174217,90 i więcej,3261,Koszalin,2021,Y_GE85,164,Koszalin
174218,90 i więcej,3262,Szczecin,2021,Y_GE85,677,Szczecin


In [160]:
total_df.query("powiat_nazwa == 'Warszawa'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
16185,0 - 4,1465,Warszawa,2002,Y_LT5,0,Warszawa
16583,5 - 9,1465,Warszawa,2002,Y5-9,0,Warszawa
16981,10 - 14,1465,Warszawa,2002,Y10-14,0,Warszawa
17379,15 - 19,1465,Warszawa,2002,Y15-19,0,Warszawa
17777,20 - 24,1465,Warszawa,2002,Y20-24,0,Warszawa
...,...,...,...,...,...,...,...
172421,70 - 74,1465,Warszawa,2021,Y70-74,1251,Warszawa
172818,75 - 79,1465,Warszawa,2021,Y75-79,1264,Warszawa
173215,80 - 84,1465,Warszawa,2021,Y80-84,1815,Warszawa
173612,85 - 89,1465,Warszawa,2021,Y_GE85,2684,Warszawa


In [161]:
total_df.loc[((total_df['name_normalized'] == 'Warszawa' ) & (total_df['year'] == '2002')), 'deaths'] = total_df.loc[((total_df['name_normalized'] == 'Warszawa' ) & (total_df['year'] == '2002')), 'deaths'].add(total_df.loc[((total_df['name_normalized'] == 'warszawski' ) & (total_df['year'] == '2002')), 'deaths'].values)

In [162]:
total_df.query("powiat_nazwa == 'Warszawa' and year == '2002'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
16185,0 - 4,1465,Warszawa,2002,Y_LT5,32,Warszawa
16583,5 - 9,1465,Warszawa,2002,Y5-9,9,Warszawa
16981,10 - 14,1465,Warszawa,2002,Y10-14,11,Warszawa
17379,15 - 19,1465,Warszawa,2002,Y15-19,12,Warszawa
17777,20 - 24,1465,Warszawa,2002,Y20-24,21,Warszawa
18175,25 - 29,1465,Warszawa,2002,Y25-29,19,Warszawa
18573,30 - 34,1465,Warszawa,2002,Y30-34,30,Warszawa
18971,35 - 39,1465,Warszawa,2002,Y35-39,41,Warszawa
19369,40 - 44,1465,Warszawa,2002,Y40-44,90,Warszawa
19767,45 - 49,1465,Warszawa,2002,Y45-49,225,Warszawa


In [163]:
total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
16173,0 - 4,1431,warszawski,2002,Y_LT5,32,warszawski
16571,5 - 9,1431,warszawski,2002,Y5-9,9,warszawski
16969,10 - 14,1431,warszawski,2002,Y10-14,11,warszawski
17367,15 - 19,1431,warszawski,2002,Y15-19,12,warszawski
17765,20 - 24,1431,warszawski,2002,Y20-24,21,warszawski
18163,25 - 29,1431,warszawski,2002,Y25-29,19,warszawski
18561,30 - 34,1431,warszawski,2002,Y30-34,28,warszawski
18959,35 - 39,1431,warszawski,2002,Y35-39,41,warszawski
19357,40 - 44,1431,warszawski,2002,Y40-44,89,warszawski
19755,45 - 49,1431,warszawski,2002,Y45-49,224,warszawski


In [164]:
rows_to_drop = total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

In [165]:
total_df.drop(rows_to_drop.index, inplace=True)

In [166]:
total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized


In [167]:
population_df.query("name_normalized == 'warszawski' and year == '2002'")


,id,name,year,population,powiat_numer,age_group,name_normalized


In [168]:
total_df = total_df.rename(columns={'wiek_eurostat':'age_group'})

In [169]:
merged_df = get_CMR_table(population_df, total_df)

In [170]:
merged_df

,id,name,year,population,powiat_numer,age_group,name_normalized_pop,wiek,powiat_nazwa,deaths,name_normalized_tot
0,011212001000,Powiat bocheński,2000,3021,1201,Y_LT5,bocheński,0 - 4,bocheński,3,bocheński
1,011212001000,Powiat bocheński,2001,2915,1201,Y_LT5,bocheński,0 - 4,bocheński,3,bocheński
2,011212001000,Powiat bocheński,2002,2820,1201,Y_LT5,bocheński,0 - 4,bocheński,5,bocheński
3,011212001000,Powiat bocheński,2003,2797,1201,Y_LT5,bocheński,0 - 4,bocheński,2,bocheński
4,011212001000,Powiat bocheński,2004,2749,1201,Y_LT5,bocheński,0 - 4,bocheński,1,bocheński
...,...,...,...,...,...,...,...,...,...,...,...
154649,071427338000,Powiat żyrardowski,2019,1067,1438,Y_GE85,żyrardowski,90 i więcej,żyrardowski,71,żyrardowski
154650,071427338000,Powiat żyrardowski,2020,1040,1438,Y_GE85,żyrardowski,85 - 89,żyrardowski,108,żyrardowski
154651,071427338000,Powiat żyrardowski,2020,1040,1438,Y_GE85,żyrardowski,90 i więcej,żyrardowski,115,żyrardowski
154652,071427338000,Powiat żyrardowski,2021,1004,1438,Y_GE85,żyrardowski,85 - 89,żyrardowski,109,żyrardowski


In [171]:
grouped_df = pd.DataFrame(merged_df.groupby(['name_normalized_tot', 'powiat_numer', 'year', 'age_group'], as_index=False).sum(['population', 'deaths']))

In [172]:
grouped_df

,name_normalized_tot,powiat_numer,year,age_group,population,deaths
0,Biała Podlaska,661,2000,Y10-14,2368,0
1,Biała Podlaska,661,2000,Y15-19,2991,3
2,Biała Podlaska,661,2000,Y20-24,2311,1
3,Biała Podlaska,661,2000,Y25-29,2047,2
4,Biała Podlaska,661,2000,Y30-34,1895,2
...,...,...,...,...,...,...
147059,żywiecki,2417,2021,Y70-74,4023,126
147060,żywiecki,2417,2021,Y75-79,2498,108
147061,żywiecki,2417,2021,Y80-84,2119,163
147062,żywiecki,2417,2021,Y_GE85,4348,418


In [174]:
cmr_total_df = pd.read_csv('../results/CMR_total_females_Y.csv')

In [175]:
cmr_total_df

,id,name,year,population,powiat_numer,name_normalized_pop,wiek,powiat_nazwa,wiek_eurostat,deaths,name_normalized_tot,CMR
0,11212001000,Powiat bocheński,2000,49494,1201,bocheński,Ogółem,bocheński,TOTAL,425,bocheński,858.689942
1,11212001000,Powiat bocheński,2001,49594,1201,bocheński,Ogółem,bocheński,TOTAL,417,bocheński,840.827519
2,11212001000,Powiat bocheński,2002,49832,1201,bocheński,Ogółem,bocheński,TOTAL,427,bocheński,856.879114
3,11212001000,Powiat bocheński,2003,50118,1201,bocheński,Ogółem,bocheński,TOTAL,386,bocheński,770.182370
4,11212001000,Powiat bocheński,2004,50970,1201,bocheński,Ogółem,bocheński,TOTAL,409,bocheński,802.432804
...,...,...,...,...,...,...,...,...,...,...,...,...
8331,71427338000,Powiat żyrardowski,2017,39689,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,460,żyrardowski,1159.011313
8332,71427338000,Powiat żyrardowski,2018,39587,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,457,żyrardowski,1154.419380
8333,71427338000,Powiat żyrardowski,2019,39518,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,463,żyrardowski,1171.617997
8334,71427338000,Powiat żyrardowski,2020,39519,1438,żyrardowski,Ogółem,żyrardowski,TOTAL,542,żyrardowski,1371.492194


In [176]:
cmr_total_copy_df = cmr_total_df.copy()

In [177]:
cmr_total_copy_df['year'] = cmr_total_copy_df['year'].astype('str')
cmr_total_copy_df['powiat_numer'] = cmr_total_copy_df['powiat_numer'].astype('str')
cmr_total_copy_df = cmr_total_copy_df.rename(columns={'wiek_eurostat':'age_group'})

In [178]:
FINAL_DF = pd.concat([cmr_total_copy_df[grouped_df.columns], grouped_df])

In [179]:
FINAL_DF

,name_normalized_tot,powiat_numer,year,age_group,population,deaths
0,bocheński,1201,2000,TOTAL,49494,425
1,bocheński,1201,2001,TOTAL,49594,417
2,bocheński,1201,2002,TOTAL,49832,427
3,bocheński,1201,2003,TOTAL,50118,386
4,bocheński,1201,2004,TOTAL,50970,409
...,...,...,...,...,...,...
147059,żywiecki,2417,2021,Y70-74,4023,126
147060,żywiecki,2417,2021,Y75-79,2498,108
147061,żywiecki,2417,2021,Y80-84,2119,163
147062,żywiecki,2417,2021,Y_GE85,4348,418


In [180]:
FINAL_DF.query('name_normalized_tot == "Warszawa" and age_group == "Y_GE85"')

,name_normalized_tot,powiat_numer,year,age_group,population,deaths
20968,Warszawa,1465,2002,Y_GE85,32418,2654
20986,Warszawa,1465,2003,Y_GE85,31248,2673
21004,Warszawa,1465,2004,Y_GE85,32088,2546
21022,Warszawa,1465,2005,Y_GE85,33708,2611
21040,Warszawa,1465,2006,Y_GE85,35520,2532
21058,Warszawa,1465,2007,Y_GE85,38376,2579
21076,Warszawa,1465,2008,Y_GE85,42004,2741
21094,Warszawa,1465,2009,Y_GE85,44946,3010
21112,Warszawa,1465,2010,Y_GE85,46314,3047
21130,Warszawa,1465,2011,Y_GE85,49590,3250


In [181]:
FINAL_DF['CMR'] = FINAL_DF['deaths']/FINAL_DF['population'] * 100000

In [182]:
FINAL_DF

,name_normalized_tot,powiat_numer,year,age_group,population,deaths,CMR
0,bocheński,1201,2000,TOTAL,49494,425,858.689942
1,bocheński,1201,2001,TOTAL,49594,417,840.827519
2,bocheński,1201,2002,TOTAL,49832,427,856.879114
3,bocheński,1201,2003,TOTAL,50118,386,770.182370
4,bocheński,1201,2004,TOTAL,50970,409,802.432804
...,...,...,...,...,...,...,...
147059,żywiecki,2417,2021,Y70-74,4023,126,3131.991051
147060,żywiecki,2417,2021,Y75-79,2498,108,4323.458767
147061,żywiecki,2417,2021,Y80-84,2119,163,7692.307692
147062,żywiecki,2417,2021,Y_GE85,4348,418,9613.615455


In [187]:
FINAL_DF.query('powiat_numer == "1201" and age_group == "TOTAL"')

,name_normalized_tot,powiat_numer,year,age_group,population,deaths,CMR
0,bocheński,1201,2000,TOTAL,49494,425,858.689942
1,bocheński,1201,2001,TOTAL,49594,417,840.827519
2,bocheński,1201,2002,TOTAL,49832,427,856.879114
3,bocheński,1201,2003,TOTAL,50118,386,770.182370
4,bocheński,1201,2004,TOTAL,50970,409,802.432804
5,bocheński,1201,2005,TOTAL,51170,429,838.381864
6,bocheński,1201,2006,TOTAL,51440,408,793.157076
7,bocheński,1201,2007,TOTAL,51680,441,853.328173
8,bocheński,1201,2008,TOTAL,51849,437,842.832070
9,bocheński,1201,2009,TOTAL,52118,457,876.856364


In [183]:
FINAL_DF.to_csv('../results/ALL_CMR_FEMALES_Y.csv', index=False)

# Dla męzczyzn

In [8]:
age_groups = """Y_LT5	72301
Y5-9	72302
Y10-14	72303
Y15-19	72304
Y20-24	47711
Y25-29	47736
Y30-34	47724
Y35-39	47712
Y40-44	47725
Y45-49	47728
Y50-54	47706
Y55-59	47715
Y60-64	47721
Y65-69	72243
Y70-74	76018
Y75-79	76019
Y80-84	76020
Y_GE85	76021"""

# Split the data into lines and then split each line into key and value
lines = age_groups.split("\n")
keys = []
values = []

for line in lines:
    key, value = line.split()
    keys.append(key)
    values.append(int(value))

# Create a dictionary from the keys and values lists
age_dict = dict(zip(keys, values))
print(age_dict)

{'Y_LT5': 72301, 'Y5-9': 72302, 'Y10-14': 72303, 'Y15-19': 72304, 'Y20-24': 47711, 'Y25-29': 47736, 'Y30-34': 47724, 'Y35-39': 47712, 'Y40-44': 47725, 'Y45-49': 47728, 'Y50-54': 47706, 'Y55-59': 47715, 'Y60-64': 47721, 'Y65-69': 72243, 'Y70-74': 76018, 'Y75-79': 76019, 'Y80-84': 76020, 'Y_GE85': 76021}


In [9]:
population_df = pd.DataFrame()

for k, v in age_dict.items():
    current_df = pd.DataFrame()
    page_0 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=0'
    page_1 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=1'
    page_2 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=2'
    page_3 = f'https://bdl.stat.gov.pl/api/v1/data/by-variable/{v}?unit-level=5&year=2000&year=2001&year=2002&year=2003&year=2004&year=2005&year=2006&year=2007&year=2008&year=2009&year=2010&year=2011&year=2012&year=2013&year=2014&year=2015&year=2016&year=2017&year=2018&year=2019&year=2020&year=2021&page-size=100&page=3'
    df_0 = get_df(page_0)
    df_1 = get_df(page_1)
    df_2 = get_df(page_2)
    df_3 = get_df(page_3)
    dfs = [df_0, df_1, df_2, df_3]
    current_df = pd.concat(dfs, axis=0)
    current_df['age_group'] = str(k)
    population_df = pd.concat([population_df, current_df], axis=0)

In [10]:
population_df

,id,name,year,value,powiat_numer,age_group
0,011212001000,Powiat bocheński,2000,3211,1201,Y_LT5
1,011212001000,Powiat bocheński,2001,3105,1201,Y_LT5
2,011212001000,Powiat bocheński,2002,3022,1201,Y_LT5
3,011212001000,Powiat bocheński,2003,2907,1201,Y_LT5
4,011212001000,Powiat bocheński,2004,2839,1201,Y_LT5
...,...,...,...,...,...,...
1615,071427338000,Powiat żyrardowski,2017,391,1438,Y_GE85
1616,071427338000,Powiat żyrardowski,2018,393,1438,Y_GE85
1617,071427338000,Powiat żyrardowski,2019,369,1438,Y_GE85
1618,071427338000,Powiat żyrardowski,2020,352,1438,Y_GE85


In [11]:
population_df = transform_population_df(population_df=population_df)

In [12]:
deaths_path = "/Users/nelatoma/Documents/icm/magisterka/zgony/MAGISTERKA_OFFICIAL/results/all_poviats_stats_males.csv"

In [13]:
total_df = get_total_df(deaths_path=deaths_path)

In [14]:
total_df

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
392,0 - 4,201,bolesławiecki,2000,Y_LT5,3,bolesławiecki
393,0 - 4,202,dzierżoniowski,2000,Y_LT5,6,dzierżoniowski
394,0 - 4,203,głogowski,2000,Y_LT5,4,głogowski
395,0 - 4,204,górowski,2000,Y_LT5,2,górowski
396,0 - 4,205,jaworski,2000,Y_LT5,4,jaworski
...,...,...,...,...,...,...,...
174215,90 i więcej,3217,wałecki,2021,Y_GE85,19,wałecki
174216,90 i więcej,3218,łobeski,2021,Y_GE85,20,łobeski
174217,90 i więcej,3261,Koszalin,2021,Y_GE85,55,Koszalin
174218,90 i więcej,3262,Szczecin,2021,Y_GE85,279,Szczecin


In [15]:
total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
16173,0 - 4,1431,warszawski,2002,Y_LT5,38,warszawski
16571,5 - 9,1431,warszawski,2002,Y5-9,7,warszawski
16969,10 - 14,1431,warszawski,2002,Y10-14,11,warszawski
17367,15 - 19,1431,warszawski,2002,Y15-19,38,warszawski
17765,20 - 24,1431,warszawski,2002,Y20-24,78,warszawski
18163,25 - 29,1431,warszawski,2002,Y25-29,65,warszawski
18561,30 - 34,1431,warszawski,2002,Y30-34,90,warszawski
18959,35 - 39,1431,warszawski,2002,Y35-39,89,warszawski
19357,40 - 44,1431,warszawski,2002,Y40-44,234,warszawski
19755,45 - 49,1431,warszawski,2002,Y45-49,449,warszawski


In [16]:
total_df.loc[((total_df['name_normalized'] == 'Warszawa' ) & (total_df['year'] == '2002')), 'deaths'] = total_df.loc[((total_df['name_normalized'] == 'Warszawa' ) & (total_df['year'] == '2002')), 'deaths'].add(total_df.loc[((total_df['name_normalized'] == 'warszawski' ) & (total_df['year'] == '2002')), 'deaths'].values)

In [17]:
total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

,wiek,powiat_numer,powiat_nazwa,year,wiek_eurostat,deaths,name_normalized
16173,0 - 4,1431,warszawski,2002,Y_LT5,38,warszawski
16571,5 - 9,1431,warszawski,2002,Y5-9,7,warszawski
16969,10 - 14,1431,warszawski,2002,Y10-14,11,warszawski
17367,15 - 19,1431,warszawski,2002,Y15-19,38,warszawski
17765,20 - 24,1431,warszawski,2002,Y20-24,78,warszawski
18163,25 - 29,1431,warszawski,2002,Y25-29,65,warszawski
18561,30 - 34,1431,warszawski,2002,Y30-34,90,warszawski
18959,35 - 39,1431,warszawski,2002,Y35-39,89,warszawski
19357,40 - 44,1431,warszawski,2002,Y40-44,234,warszawski
19755,45 - 49,1431,warszawski,2002,Y45-49,449,warszawski


In [18]:
rows_to_drop = total_df.query("powiat_nazwa == 'warszawski' and year == '2002'")

total_df.drop(rows_to_drop.index, inplace=True)

In [19]:
total_df = total_df.rename(columns={'wiek_eurostat':'age_group'})


In [20]:
merged_df = get_CMR_table(population_df, total_df)


In [21]:
merged_df

,id,name,year,population,powiat_numer,age_group,name_normalized_pop,wiek,powiat_nazwa,deaths,name_normalized_tot
0,011212001000,Powiat bocheński,2000,3211,1201,Y_LT5,bocheński,0 - 4,bocheński,5,bocheński
1,011212001000,Powiat bocheński,2001,3105,1201,Y_LT5,bocheński,0 - 4,bocheński,6,bocheński
2,011212001000,Powiat bocheński,2002,3022,1201,Y_LT5,bocheński,0 - 4,bocheński,3,bocheński
3,011212001000,Powiat bocheński,2003,2907,1201,Y_LT5,bocheński,0 - 4,bocheński,4,bocheński
4,011212001000,Powiat bocheński,2004,2839,1201,Y_LT5,bocheński,0 - 4,bocheński,2,bocheński
...,...,...,...,...,...,...,...,...,...,...,...
154649,071427338000,Powiat żyrardowski,2019,369,1438,Y_GE85,żyrardowski,90 i więcej,żyrardowski,36,żyrardowski
154650,071427338000,Powiat żyrardowski,2020,352,1438,Y_GE85,żyrardowski,85 - 89,żyrardowski,60,żyrardowski
154651,071427338000,Powiat żyrardowski,2020,352,1438,Y_GE85,żyrardowski,90 i więcej,żyrardowski,38,żyrardowski
154652,071427338000,Powiat żyrardowski,2021,365,1438,Y_GE85,żyrardowski,85 - 89,żyrardowski,41,żyrardowski


In [22]:
grouped_df = pd.DataFrame(merged_df.groupby(['name_normalized_tot', 'powiat_numer', 'year', 'age_group'], as_index=False).sum(['population', 'deaths']))


In [23]:
grouped_df

,name_normalized_tot,powiat_numer,year,age_group,population,deaths
0,Biała Podlaska,661,2000,Y10-14,2880,0
1,Biała Podlaska,661,2000,Y15-19,2942,1
2,Biała Podlaska,661,2000,Y20-24,2390,4
3,Biała Podlaska,661,2000,Y25-29,2151,6
4,Biała Podlaska,661,2000,Y30-34,1785,2
...,...,...,...,...,...,...
147059,żywiecki,2417,2021,Y70-74,3126,171
147060,żywiecki,2417,2021,Y75-79,1561,144
147061,żywiecki,2417,2021,Y80-84,1123,151
147062,żywiecki,2417,2021,Y_GE85,1488,178


In [24]:
cmr_total_df = pd.read_csv('../results/CMR_total_males_Y.csv')

In [25]:
cmr_total_copy_df = cmr_total_df.copy()

In [26]:
cmr_total_copy_df['year'] = cmr_total_copy_df['year'].astype('str')
cmr_total_copy_df['powiat_numer'] = cmr_total_copy_df['powiat_numer'].astype('str')
cmr_total_copy_df = cmr_total_copy_df.rename(columns={'wiek_eurostat':'age_group'})

In [27]:
FINAL_DF = pd.concat([cmr_total_copy_df[grouped_df.columns], grouped_df])


In [28]:
FINAL_DF['CMR'] = FINAL_DF['deaths']/FINAL_DF['population'] * 100000


In [30]:
FINAL_DF.query('age_group == "Y40-44"')

,name_normalized_tot,powiat_numer,year,age_group,population,deaths,CMR
6,Biała Podlaska,661,2000,Y40-44,2469,18,729.040097
20,Biała Podlaska,661,2001,Y40-44,2359,22,932.598559
34,Biała Podlaska,661,2002,Y40-44,2231,7,313.760645
52,Biała Podlaska,661,2003,Y40-44,2114,7,331.125828
70,Biała Podlaska,661,2004,Y40-44,2006,12,598.205384
...,...,...,...,...,...,...,...
146980,żywiecki,2417,2017,Y40-44,5711,18,315.181229
146998,żywiecki,2417,2018,Y40-44,5760,23,399.305556
147016,żywiecki,2417,2019,Y40-44,5776,14,242.382271
147034,żywiecki,2417,2020,Y40-44,5862,15,255.885363


In [ ]:
FINAL_DF.to_csv('../results/ALL_CMR_MALES_Y.csv', index=False)